# Objectives

In this Assignment, you will demonstrate the data visualization skills you learned by completing this course.   Dashboard is based on the concept of demonstrating US Domestic Airline Flights Performance and Yearly average flight delay statistics for a given year ( 2005 to 2020). 

# Downloading and Prepping Data

If you are using local jupyter lab, then  add these lines in your code:

In [1]:
"""
!pip install dash
!pip install dash==1.19.0
!pip install jupyter_dash
!pip install --upgrade plotly
!pip install jupyter-dash
"""

'\n!pip install dash\n!pip install dash==1.19.0\n!pip install jupyter_dash\n!pip install --upgrade plotly\n!pip install jupyter-dash\n'

## TASK 1 - Read the data

Let's start with

* Importing necessary libraries
* Reading the data

In [2]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
#import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output

# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

# List of years 
year_list=[i for i in range(2005, 2021, 1)]

## TASK 2 - Create dash application and get the layout skeleton

#### Is the dashboard titled "US Domestic Airline Flights Performance" with center alignment?

What is expected?

Make sure :
- The dash application has the title as `US Domestic Airline Flights Performance` 
- The heading is center aligned.

In [3]:
# Create a dash application
app=JupyterDash(__name__) #This line used for JupyterLab

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions=True

# Build dash app layout
app.layout=html.Div(children=[html.H1("US Domestic Airline Flights Performance",
                                     style={'textAlign': 'center', 'color': '#503D36','font-size': 24}),
                              
                              #outer division starts
                              html.Div([
                                  
                                  html.Div(html.H2("Report Type:",style={'margin-right': '2em'})),
                                  
                                  dcc.Dropdown(id="dropdown-menu1",
                                               options=[{'label': 'Yearly Airline Performance Report', 'value':"yapr"},
                                                        {'label': 'Yearly Average Flight Delay Statistics', 'value':"yafds"}
                                                        ],placeholder='Select a Report Type',style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'text-align-last':'center'},
                                                     value="yapr"),
                                  
                                  html.Div(html.H2("Choose Year:",style={'margin-right': '2em'})),
                                  dcc.Dropdown(id="dropdown-menu2",
                                               options=[{'label': i, 'value': i} for i in year_list],
                                               placeholder='Select a Year',
                                               style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'text-align-last':'center'}, 
                                               value="year5"),
                                                                  html.Div([
                                        html.Div(dcc.Graph(id="carrier-plot")),
                                        html.Div(dcc.Graph(id="weather-plot"))
                                ], style={'display': 'flex'}),
    
                                html.Div([
                                        html.Div(dcc.Graph(id="nas-plot")),
                                        html.Div(dcc.Graph(id="security-plot"))
                                ], style={'display': 'flex'}),
                                
                                html.Div(dcc.Graph(id="late-plot"), style={'width':'65%'})

                              ]) #outer division ends
                             
                             
                             
                        ])#layout ends

#### Is the dashboard containing two dropdown?

What is expected?

Below are the 2 dropdown menus title and options.

Dropdown menu 1 Title: `Report type` 

Options:

- Yearly airline performance report 

- Yearly average flight delay statistics


#### Does `Choose Year` dropdown contain year from 2005 to 2020?

Dropdown menu 2 Title: `Choose Year`

Options: Display from 2005 to 2020

In [4]:
""" Compute_info function description

This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments:
    airline_data: Input airline data.
    entered_year: Input year for which computation needs to be performed.
    
Returns:
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.

"""

def compute_info(airline_data, entered_year):
    
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

# Callback decorator
@app.callback( [
               Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'), 
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')
               ],
               Input(component_id='input-year', component_property='value'))
# Computation to callback function and return graph
def get_graph(entered_year):
    
    # Compute required information for creating graph from the data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)
            
    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrier delay time (minutes) by airline')
    
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
    
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
    
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
    
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]


   
if __name__ == '__main__':
    #app.run_server(mode="inline")
    
        
    #For JupyterLab
    app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, debug=True,dev_tools_hot_reload =True, threaded=True)